### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
- mushroomの50サンプル分

### yのサンプリング
- 条件に合うまでサンプルを作り続ける

### 改訂(2020/08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である
- 正確な分布を作ることが必要なので、yを全列挙する必要がある。

### 改訂（2020/08/18）
- mushroom_l データを更新（sumが8になるようにした）

### 改訂（2020/08/20）
- make_y関数のバグを発見

In [204]:
#!python3 --version

In [205]:
#!pip3 install dwave-neal

In [206]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import itertools

%matplotlib inline
import matplotlib.pyplot as plt

In [207]:
#X_mushroom = pd.read_csv("../input/SNP_df1010.csv", sep=',', index_col=0).iloc[:10, :10]
#y_mushroom = pd.read_csv('../input/y10.csv', sep=',', index_col=0)['y'][:10]

X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0).iloc[:10, :]
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21'][:10]

In [208]:
X_mushroom

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
7253,0,1,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
2721,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0
2274,0,1,1,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
4814,1,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
6604,1,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
7501,1,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
4238,1,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1003,1,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
3875,0,1,0,0,0,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
5296,1,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0


In [209]:
y_mushroom

7253    1
2721    1
2274    1
4814    0
6604    0
7501    0
4238    0
1003    0
3875    1
5296    0
6900    1
6405    0
2710    1
653     0
2883    1
4634    1
Name: 21, dtype: int64

In [210]:
#X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0)
#y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [211]:
num_samples

16

In [212]:
num_cols

117

In [213]:
y_sum

8

In [214]:
2**num_samples

65536

In [215]:
y_mushroom

7253    1
2721    1
2274    1
4814    0
6604    0
7501    0
4238    0
1003    0
3875    1
5296    0
6900    1
6405    0
2710    1
653     0
2883    1
4634    1
Name: 21, dtype: int64

-----

In [220]:
def make_y(minP_cols):
    y_list = []
    y_list_all = itertools.product('01', repeat=num_samples)

    for y_tuple in y_list_all:
        y_list_list = [int(n) for n in list(y_tuple)]
        y = pd.Series(y_list_list)

        if sum(y) - y_sum != 0:
            continue

        #print(y)
        if len(minP_cols) == 0:
            y_list.append(y) #pd.Seriesを要素として足している
        else:
            for i in minP_cols: #選択されたyはすべてのiについて👇の条件を満たす必要がある
                print(i)
                print(np.dot(X_mushroom[i], y))
                print(np.dot(X_mushroom[i], y_mushroom))
                print('--------------------')
                if np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom) != 0:
                    break      
            else:
                y_list.append(y) #pd.Seriesを要素として足している
    return y_list

In [221]:
minP_cols = []
servey_num_col_list = columns

while len(servey_num_col_list) > 0:  
    y_list = make_y(minP_cols)
    print(len(y_list)) ##
    
    if len(y_list) == 0:
        print(minP_cols)
        print('--------------')
        break
        
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        
        for y in y_list:       
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k > ori_ti]) #2020/08/20変更
        p[i] = count_better_t / len(y_list)
        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    print(minP_keys) ##
    
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
    print(servey_num_col_list)

12870
['1', '4', '7', '10', '11', '15', '16', '17', '19', '22', '23', '25', '26', '27', '30', '31', '32', '34', '35', '36', '37', '38', '41', '43', '44', '45', '46', '48', '49', '51', '56', '57', '60', '63', '64', '65', '66', '67', '70', '72', '73', '74', '76', '77', '79', '80', '81', '82', '83', '85', '86', '87', '88', '89', '91', '93', '94', '95', '96', '97', '100', '101', '102', '103', '105', '106', '107', '108', '111', '115']
['0', '2', '3', '5', '6', '8', '9', '12', '13', '14', '18', '20', '21', '24', '28', '29', '33', '39', '40', '42', '47', '50', '52', '53', '54', '55', '58', '59', '61', '62', '68', '69', '71', '75', '78', '84', '90', '92', '98', '99', '104', '109', '110', '112', '113', '114', '116']
1
5
8
--------------------
1
4
8
--------------------
1
5
8
--------------------
1
4
8
--------------------
1
5
8
--------------------
1
4
8
--------------------
1
5
8
--------------------
1
4
8
--------------------
1
4
8
--------------------
1
4
8
--------------------
1
5
8
-------

In [12]:
print(y_list[0])

0     1
1     1
2     1
3     0
4     0
5     0
6     0
7     0
8     1
9     0
10    1
11    0
12    1
13    0
14    1
15    1
dtype: int64


In [13]:
list(y_mushroom)==list(y_list[0])

True

In [14]:
y_list_all = itertools.product('01', repeat=num_samples)

ttt = 0
for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum == 0:
        if np.dot(X_mushroom['1'], y) - np.dot(X_mushroom['1'], y_mushroom) == 0:
            print(y)
            ttt += 1
ttt

0     1
1     1
2     1
3     0
4     0
5     0
6     0
7     0
8     1
9     0
10    1
11    0
12    1
13    0
14    1
15    1
dtype: int64


1

-----

### t^0のみ

In [123]:
#t^0に当てはまるyの数を知る
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum == 0:
        y_list.append(y) #type(y)はSeries
len(y_list)

24310

In [53]:
ori_t1 = int(np.dot(X_mushroom['1'], y_mushroom)) #特徴量iを表すp
#print(ori_t1) #8
for y in y_list:
    #print(y)
    if list(y)==list(y_mushroom):
        print(y)
    if int(np.dot(X_mushroom['1'], y))==ori_t1:
        print(y)
        print('---------')#originalのyしか該当するyはないようだ

0     0
1     1
2     1
3     1
4     0
5     1
6     0
7     1
8     0
9     1
10    0
11    1
12    0
13    0
14    0
15    1
dtype: int64
0     0
1     1
2     1
3     1
4     0
5     1
6     0
7     1
8     0
9     1
10    0
11    1
12    0
13    0
14    0
15    1
dtype: int64
---------


In [55]:
0.000078*12870  #約1(1.00386)

1.00386

In [108]:
plt.rcParams["figure.figsize"] = [100, 150]  # 図の縦横のサイズ([横(inch),縦(inch)])
plt.rcParams["figure.autolayout"] = True   # レイアウトの自動調整を利用するかどうか
plt.rcParams["figure.subplot.wspace"] = 0.10# 図が複数枚ある時の左右との余白
plt.rcParams["figure.subplot.hspace"] = 0.10# 図が複数枚ある時の上下との余白

fig, ax = plt.subplots(int(num_cols/3)+1, 3)

minP_cols = []
servey_num_col_list = columns

enul_p = {}
for index, i in enumerate(servey_num_col_list):
    
    ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

    ti_dic = {} #{内積の値:該当数}という辞書。

    for y in y_list:       
        if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
            ti_dic[int(np.dot(X_mushroom[i], y))] += 1
        else:
            ti_dic[int(np.dot(X_mushroom[i], y))] = 1

    ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
    franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
    enul_p[i] = franc/len(y_list)
    
    n_row = int(index%(int(num_cols/3)+1))#行
    
    #列
    if index < int(num_cols/3)+1:
        n_col = 0
    elif index < (int(num_cols/3)+1)*2:
        n_col = 1
    else:
        n_col = 2
    
    #ヒストグラムを出力
    ax[n_row, n_col].set_title(i, fontsize=15)
    ax[n_row, n_col].bar([j[0] for j in ti_list], [j[1] for j in ti_list])
    ax[n_row, n_col].scatter(ori_ti, 100, s=200, marker='D')
    ax[n_row, n_col].set_xlabel('inner product size', fontsize=10)
    ax[n_row, n_col].set_ylabel('number', rotation=90, fontsize=10, labelpad=20)
    
#図が重ならないようにする
fig.tight_layout()

fig.subplots_adjust(bottom=0, left=0, top=0.1, right=0.1, wspace=0)
    
#全体に一つの題名を表示
fig.suptitle('inner product size and its number using all y which meet conditions', fontsize=15)

#図を表示
#plt.show()

#fig.savefig('../output/mushrooml/enul_p.png')

enul_p = pd.DataFrame.from_dict(enul_p, orient='index').T
print(enul_p)

enul_p.to_csv('../output/mushrooml/enul_p.csv')

     0         1         2    3    4         5    6    7    8    9  ...  107  \
0  1.0  0.000078  0.858974  1.0  1.0  0.304196  0.5  1.0  1.0  0.5  ...  1.0   

   108  109       110  111       112       113  114  115  116  
0  1.0  0.5  0.934033  0.5  0.157343  0.715385  1.0  1.0  1.0  

[1 rows x 117 columns]


In [109]:
enul_p

,0,1,2,3,4,5,6,7,8,9,...,107,108,109,110,111,112,113,114,115,116
0,1.0,0.000078,0.858974,1.0,1.0,0.304196,0.5,1.0,1.0,0.5,...,1.0,1.0,0.5,0.934033,0.5,0.157343,0.715385,1.0,1.0,1.0


In [110]:
#enul_pにおける各列の最小さい値
enul_p.idxmin(axis=1)

0    1
dtype: object

In [37]:
np.dot(X_mushroom['1'], y_mushroom)

8

In [63]:
y_mushroom

1064    0
2812    1
8073    1
2262    1
4290    0
1032    1
6145    0
7022    1
4495    0
8063    1
7061    0
1140    1
4349    0
6868    0
4214    0
333     1
Name: 21, dtype: int64

In [65]:
np.dot(X_mushroom['1'], pd.Series([0, 1,1,1,0,1,0,1,0,1,0,1,0, 0, 0, 1])) 

8

In [ ]:
np.dot(X_mushroom['27'], y) - np.dot(X_mushroom['27'], y_mushroom)

In [95]:
#選ばれるyがあるかどうかの確認（p値とは無関係）
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum != 0:
        continue

    if np.dot(X_mushroom['1'], y) - np.dot(X_mushroom['1'], y_mushroom) == 0:
        if np.dot(X_mushroom['27'], y) - np.dot(X_mushroom['27'], y_mushroom) == 0:
            print(y)

0     0
1     1
2     1
3     1
4     0
5     1
6     0
7     1
8     0
9     1
10    0
11    1
12    0
13    0
14    0
15    1
dtype: int64


In [94]:
#選ばれるyがあるかどうかの確認（p値とは無関係）
y_list = []
y_list_all = itertools.product('01', repeat=num_samples)

for y_tuple in y_list_all:
    y_list_list = [int(n) for n in list(y_tuple)]
    y = pd.Series(y_list_list)

    if sum(y) - y_sum != 0:
        continue

    if np.dot(X_mushroom['1'], y) - np.dot(X_mushroom['1'], y_mushroom) == 0:
        print(y)

0     0
1     1
2     1
3     1
4     0
5     1
6     0
7     1
8     0
9     1
10    0
11    1
12    0
13    0
14    0
15    1
dtype: int64


In [86]:
np.dot(X_mushroom['2'], y) - np.dot(X_mushroom['2'], y_mushroom)

0

In [85]:
np.dot(X_mushroom['27'], y) - np.dot(X_mushroom['27'], y_mushroom)

0